In [ ]:
import matplotlib.pyplot as plt

labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
fracs = [15, 30, 45, 10]

fig, axs = plt.subplots()

# axs.pie(fracs, labels=labels, autopct='%1.1f%%', shadow=True)
axs.pie(fracs, labels=labels, autopct='%.0f%%', shadow=True, explode=(0.1, 0.1,0,0))

In [2]:
import akshare as ak



In [27]:
lrb = ak.stock_lrb_em(date="20220331")

In [26]:
lrb.to_csv("./lrb.csv")

In [30]:
lrb.loc[lrb["股票代码"] == "000858"]

,序号,股票代码,股票简称,净利润,净利润同比,营业总收入,营业总收入同比,营业总支出-营业支出,营业总支出-销售费用,营业总支出-管理费用,营业总支出-财务费用,营业总支出-营业总支出,营业利润,利润总额,公告日期
2082,2083,000858,五粮液,1.082287e+10,16.08,2.754816e+10,13.251148,5.948172e+09,2.033172e+09,9.804271e+08,-5.133086e+08,1.240055e+10,1.516849e+10,1.513640e+10,2022-04-29


In [3]:
gdp = ak.macro_china_gdp()
print(gdp)

             季度  国内生产总值-绝对值  国内生产总值-同比增长  第一产业-绝对值  第一产业-同比增长  第二产业-绝对值  \
0   2022年第1-3季度    870269.0          3.0   54779.1        4.2  350189.5   
1   2022年第1-2季度    562641.6          2.5   29137.2        5.0  228636.4   
2     2022年第1季度    270177.8          4.8   10953.8        6.0  106186.6   
3   2021年第1-4季度   1143669.7          8.1   83085.5        7.1  450904.5   
4   2021年第1-3季度    819432.3          9.8   51613.3        7.4  320254.7   
..          ...         ...          ...       ...        ...       ...   
62    2007年第1季度     57159.3         13.8    3473.0        4.1   25983.1   
63  2006年第1-4季度    219438.5         12.7   23317.0        4.8  104359.2   
64  2006年第1-3季度    155816.8         12.8   14700.4        4.7   73929.5   
65  2006年第1-2季度     99752.2         13.1    7762.7        4.8   46992.9   
66    2006年第1季度     47078.9         12.5    3012.7        4.4   21418.2   

    第二产业-同比增长  第三产业-绝对值  第三产业-同比增长  
0         3.9  465300.4        2.3  
1         3.2  304868.0  

In [4]:
div = ak.stock_history_dividend()

In [8]:
# print(div[div['代码'] == '000858'])
print(div)

          代码    名称        上市日期   累计股息  年均股息  分红次数      融资总额  融资次数
0     000550  江铃汽车  1993-12-01  202.1  6.97    47    0.0000     0
1     000541  佛山照明  1993-11-23  182.9  6.31    52    0.0000     0
2     000429  粤高速A  1998-02-20  148.9  5.96    46   16.3350     1
3     000726   鲁泰A  2000-12-25  136.7  6.22    43    9.5082     1
4     000022  深赤湾A  1993-05-05  132.3  5.09    45    0.0000     0
...      ...   ...         ...    ...   ...   ...       ...   ...
4611  688789  宏华数科  2021-07-08    0.0  0.00     0    5.0340     1
4612  688798  艾为电子  2021-08-16    0.0  0.00     0   30.3526     1
4613  688799  华纳药厂  2021-07-13    0.0  0.00     0    6.5566     1
4614  688800   瑞可达  2021-07-22    0.0  0.00     0    3.4983     1
4615  688981  中芯国际  2020-07-16    0.0  0.00     0  525.1560     1

[4616 rows x 8 columns]


In [10]:
# Annualized standard deviation (volatility) of monthly return
# 月收益率的年化标准差（波动率）
# 实证研究表明，收益率标准差（波动率）存在一定的集聚现象，
# 即高波动率和低波动率往往会各自聚集在一起，并且高波动率和低波动率聚集的时期是交替出现的。
# 红色部门显示出，所有股票均存在一定的波动集聚现象。
fig = plt.figure(figsize = (16,15))
for ii,jj in enumerate(tickers):
    plt.subplot(5,2,ii+1)
    daily_ret=stocks[jj].pct_change()
    mnthly_annu = daily_ret.resample('M').std()* np.sqrt(12)
    #plt.rcParams['figure.figsize']=[20,5]
    mnthly_annu.plot(color=color_palette[ii],label=jj)
    start_date=mnthly_annu.index[0]
    end_date=mnthly_annu.index[-1]
    plt.xticks(pd.date_range(start_date,end_date,freq='Y'),[str(y) for y in range(start_date.year+1,end_date.year+1)])
    dates=mnthly_annu[mnthly_annu>0.07].index
    for i in range(0,len(dates)-1,3):
        plt.axvspan(dates[i],dates[i+1],color=color_palette[ii],alpha=.3)
    plt.title('%s - Standard deviation of monthly rate of return'%jj,size=15)
    labs = mpatches.Patch(color=color_palette[ii],alpha=.5, label="Volatility agglomeration")
    plt.legend(handles=[labs])
plt.tight_layout()

NameError: name 'plt' is not defined